# Import module

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

Or from the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

## ImageAnalysis3 and basic modules

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py
import ast
import pandas as pd

print(os.getpid())

12912


## Chromatin_analysis_tools etc

See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [2]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

# Define folders

In [3]:
# main folder for postanalysis
postanalysis_folder = r'L:\Shiwei\postanalysis_2024\v0'
# input files for postanalysis
input_folder = os.path.join(postanalysis_folder, 'resources_from_preprocess')

# output file to be generated
output_main_folder = os.path.join(postanalysis_folder, 'locus_annotation')

output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

# make new folder if needed
make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

Use existing analysis folder: L:\Shiwei\postanalysis_2024\v0\locus_annotation\analysis.
Use existing figure folder: L:\Shiwei\postanalysis_2024\v0\locus_annotation\figures.


# Plotting parameters

In [4]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})  

# Load data relevant information

## load codebook

[merged codebook](../resources/merged_codebook.csv) as in the repository (merged for all DNA-MERFISH libraries)

In [5]:
# Load codebook 
codebook_fname = os.path.join(input_folder,'merged_codebook.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df by chr and chr_order
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)
codebook_df.head()

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,1:3742742-3759944,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1:6245958-6258969,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1:8740008-8759916,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1016,1:9627926-9637875,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1017,1:9799472-9811359,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## convert to new dataframe to store the transcirption annotation

In [6]:
# Format the chr loci name 
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)

# convert to a new dataframe and set loci name as index
loci_df = codebook_df[['name','id','chr','chr_order','library']]
loci_df['loci_name'] = list(loci_name_arr[:,0])
loci_df = loci_df.set_index ('loci_name')

loci_df.head()

C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_12912\58657782.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loci_df['loci_name'] = list(loci_name_arr[:,0])


,name,id,chr,chr_order,library
loci_name,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0.0,CTP11
chr1_6245958_6258969,1:6245958-6258969,2,1,1.0,CTP11
chr1_8740008_8759916,1:8740008-8759916,3,1,2.0,CTP11
chr1_9627926_9637875,1:9627926-9637875,1,1,3.0,CTP13
chr1_9799472_9811359,1:9799472-9811359,2,1,4.0,CTP13


## load all gene annotation from Ensembl df

annotation can be generated using the notebook below:
[external/scripts/genome_annotation/1_genomic_info_for_mop_genes_using_ensembl_transcriptome](../../../external/scripts/genome_annotation/1_genomic_info_for_mop_genes_using_ensembl_transcriptome.ipynb)


Annotation link in this repository:

[MOp_annotation_smart_seq](../../../external/resources/MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv)

[MOp_annotation_10x](../../../external/resources/MOp_10x_snRNA_chr_info_NEW_from_transcriptome_FORMAT.csv)

In [7]:
# load gene annotation (covering all genes from the SMART-seq) for chr locus
# this should be enough since we can only analyze the genes with info from SMARTseq
# also we used Ensembl for both gene2loci and loci2gene to be consistent 
external_folder = os.path.join(postanalysis_folder, 'external_resources')
gene_annotation_df = pd.read_csv(os.path.join(external_folder, 
                                              "MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv"),
                                 index_col=0)

gene_annotation_df.head()

,chr,start,end,gene_biotype,coding_strand,length,genomic_position
gene,,,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1,7533,chr7_45567794_45575327
0610009B22Rik,11,51685386,51688874,protein_coding,-1,3488,chr11_51685386_51688874
0610009E02Rik,2,26445696,26459390,processed_transcript,1,13694,chr2_26445696_26459390
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1,2512,chr11_120348678_120351190
0610010F05Rik,11,23564961,23633639,protein_coding,-1,68678,chr11_23564961_23633639


## function to add adjacent genes by defined distance

In [8]:
# Function to find genes near a list of loci provided in DataFrame 
from gene_to_loci import find_genes_near_loci

def find_genes_near_loci_for_loci_dataframe (loci_df:pd.core.frame.DataFrame, 
                                             gene_annotation_df:pd.core.frame.DataFrame, 
                                             extend_dist = 50*1000,
                                             gene_coverage_type = 'tss',
                                             key_added = None):


    """Find genes near loci for a list of loci provided in DataFrame"""
    sel_genes_list = []
    for loci_name in loci_df.index.tolist():
        sel_genes =  find_genes_near_loci (loci_name, 
                                           gene_annotation_df,
                                           extend_dist = extend_dist, 
                                           gene_coverage_type = gene_coverage_type
                                          )
        sel_genes_list.append(sel_genes)
    
    if isinstance (key_added, type(None)):
        extend_dist_name = str(int(extend_dist/1000)) + 'kb'
        key_added = f'adjacent_genes_{extend_dist_name}_{gene_coverage_type}'
    
    loci_df[key_added]=sel_genes_list
    
    return loci_df

## add adjacent genes for different distances

In [9]:
loci_df_copy = loci_df.copy(deep=True)

for _dist in [10*1000,50*1000,100*1000,200*1000,500*1000,1000*1000,1500*1000,2000*1000,]:

    loci_df_copy = find_genes_near_loci_for_loci_dataframe (loci_df_copy, 
                                            gene_annotation_df,
                                             extend_dist = _dist, # distance to extend both direction
                                             gene_coverage_type = 'tss', # distance to gene's coord of interest
                                             key_added = None)

loci_df_copy

,name,id,chr,chr_order,library,adjacent_genes_10kb_tss,adjacent_genes_50kb_tss,adjacent_genes_100kb_tss,adjacent_genes_200kb_tss,adjacent_genes_500kb_tss,adjacent_genes_1000kb_tss,adjacent_genes_1500kb_tss,adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0.0,CTP11,intergenic,intergenic,Xkr4,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6245958_6258969,1:6245958-6258969,2,1,1.0,CTP11,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740008_8759916,1:8740008-8759916,3,1,2.0,CTP11,intergenic,Gm15452,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_9627926_9637875,1:9627926-9637875,1,1,3.0,CTP13,2610203C22Rik,2610203C22Rik,2610203C22Rik; Adhfe1; Gm6161; Mybl1; Rrs1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_9799472_9811359,1:9799472-9811359,2,1,4.0,CTP13,Gm6195; Sgk3,Gm6195; Sgk3,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Vc...,1700034P13Rik; 2610203C22Rik; Gm6195; Mcmdc2; ...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,X:166247682-166259932,1059,X,60.0,CTP11,Gpm6b,Gpm6b,Gemin8; Gpm6b,Gemin8; Gm15225; Gpm6b; Ofd1; Trappc2,Egfl6; Gemin8; Gm15223; Gm15225; Gm1720; Gpm6b...,Egfl6; Gemin8; Glra2; Gm15223; Gm15225; Gm1523...,Egfl6; Fancb; Gemin8; Glra2; Gm15223; Gm15225;...,Asb11; Asb9; Bmx; Egfl6; Fancb; Gemin8; Glra2;...
chrX_167157164_167167452,X:167157164-167167452,990,X,61.0,CTP13,intergenic,Tmsb4x,Gm15232; Tmsb4x,Gm15232; Gm8814; Tlr7; Tmsb4x,Gm15230; Gm15232; Gm1720; Gm8814; Prps2; Tlr7;...,Egfl6; Gemin8; Gm15223; Gm15230; Gm15232; Gm15...,Egfl6; Frmpd4; Gemin8; Gm15223; Gm15225; Gm152...,Arhgap6; Egfl6; Frmpd4; Gemin8; Glra2; Gm15223...
chrX_168746045_168757590,X:168746045-168757590,1060,X,62.0,CTP11,intergenic,Arhgap6,Arhgap6; Msl3,Arhgap6; Frmpd4; Msl3,Amelx; Arhgap6; Frmpd4; Msl3,Amelx; Arhgap6; Frmpd4; Gm15238; Gm15240; Hccs...,Amelx; Arhgap6; Erdr1; Frmpd4; G530011O06Rik; ...,Amelx; Arhgap6; Asmt; Erdr1; Frmpd4; G530011O0...


## save the annotated file

In [10]:
# save the file 
loci_df_copy.to_csv(os.path.join(output_analysis_folder,'MERFISH_loci_adjacent_genes_tss_more_res.csv'))

The scripts above can be used to generate gene annotation as in the repository:

[MERFISH_loci_annotation](../resources/MERFISH_loci_adjacent_genes_tss_more_res.csv)

